In [1]:
from langchain_ollama import OllamaLLM
from langchain.chat_models import init_chat_model
import getpass
import os
import torch
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

llm = init_chat_model("llama3.1",model_provider = "Ollama")

In [2]:
from langchain_community.document_loaders import CSVLoader
from langchain_community.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
loader = CSVLoader(
    file_path = 'nirf_clean.csv',
    source_column="Name",
    metadata_columns=["Rank","State"],
)
docs = loader.load()
embed = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={"device": device}
    )
db = Chroma.from_documents(docs,embed,persist_directory="./chroma_db")
retiever = db.as_retriever(search_kargs={"k":3})

/Users/nilaychheda/Desktop/CodingShit/RAG/rag_env/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:

if not os.environ.get("TAVILY_API_KEY"):
    os.environ["TAVILY_API_KEY"] = getpass.getpass("Tavily API key: ")
from langchain_tavily import TavilySearch
search_tool = TavilySearch(
    max_results=5,
    topic="general"
    )

In [ ]:
""" from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.documents import Document
from accelerate import init_empty_weights
embed = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2",model_kwargs={"device": device})
response = search_tool.invoke({"query": "What was the cutoff for nit trichy cs in 2024?"})
print(response)
raw_docs = response['results']
docs = [
    Document(page_content=res['content'],metadata={"url": res['url']}) for res in raw_docs
]
db = Chroma.from_documents(docs,embed,persist_directory="chroma_db") """

In [4]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(
    memory_key="history",
    input_key="input"
    )

/var/folders/q5/g1s9s5h155s0fl0r7wf047640000gn/T/ipykernel_94515/2033394661.py:2: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


In [5]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
template = """
You are an AI agent, specifically modeled to answer questions related to various engineering enterance exams in India.
You should ask the user various questions related to his exam scores, branch preferences, interests etc to access his profile, and then reccomend him 3 colleges
You must answer all the queires wihh the context provided below:
Here is some relevant context form old database : {context}
Here is some context taken from web: {web_context}
Here is the chat history: {history}
Here is the query : {input}
"""

prompt = ChatPromptTemplate.from_template(template)

document_chain = create_stuff_documents_chain(llm,prompt)
rag_chain = create_retrieval_chain(retiever,document_chain)


In [6]:
from langchain_core.documents import Document
from accelerate import init_empty_weights

print("Hello! I am a llm agent to help with all your doubts regarding the various diffrent engineering entrance exams in India." \
"Type 'exit' to quit")
while True:
    user_input = input("\nUser: ")
    if user_input == "exit":
        print("Exiting conversation.")
        break

    db_results = retiever.invoke(user_input)
    context = "\n".join(res.page_content for res in db_results)

    web_results = search_tool.invoke({"query":user_input})
    raw = web_results['results']
    web_context = [
        Document(page_content=res['content'], metadata= {"url": res['url']}) for res in raw
    ]
    history_dict = memory.load_memory_variables({})
    history = history_dict.get("history", "")

    response = rag_chain.invoke({
        "input": user_input,
        "web_context": web_context,
        "context": context,
        "history": history,
    })
    memory.save_context({"input": user_input},{"output":response['answer']})
    print(f"AI: {response['answer']}")

Hello! I am a llm agent to help with all your doubts regarding the various diffrent engineering entrance exams in India.Type 'exit' to quit
AI: The JEE Advanced 2025 examination is scheduled to be held on Sunday, May 18, 2025. It will consist of two papers - Paper 1 from 9:00 AM to 12:00 PM IST and Paper 2 from 2:30 PM to 5:30 PM IST.

However, I'd like to know more about your interests and goals. What branch are you interested in pursuing after clearing JEE Advanced? Are you inclined towards core branches like CS or EE, or do you have a preference for Humanities or other disciplines?

Also, could you please share your expected rank and branch preferences? This will help me provide you with more tailored recommendations.
AI: Thank you for sharing your details with me.

Considering your expected rank of 5000 in JEE Advanced and your preferences for CS, ECE, or other technical branches, here are three college recommendations for you:

1. **Indian Institute of Engineering Science and Tech